In [4]:
from pathlib import Path
import sys
from pathlib import Path

# Add the parent directory of 'machine_learning' to sys.path
sys.path.append(str(Path().cwd().parent.parent.parent))

In [5]:
from machine_learning.transactions_classification.lib.data_loader import load_treated_dataset

# Load dataset

In [ ]:
import os
dataset_path = Path().cwd().parent / "data"
key = os.getenv('ENC_KEY', '').encode()
complete_dataset = load_treated_dataset(dataset_path, key)
complete_dataset

,Data,Descrição,categoria,Valor
0,2022-10-10,parc=105ponto frio.c*3483 - parc.4/5,Presentes,-172.38
1,2022-11-20,ec*pantys - parc.3/3,Autocuidado - produtos,-91.83
2,2022-11-22,milena afradique - parc.2/3,Autocuidado - serviços,-566.67
3,2022-11-25,werner coiffeur bota - parc.2/3,Autocuidado - serviços,-483.33
4,2022-11-26,nespresso - parc.2/4,Alimentação cotidiana,-103.00
...,...,...,...,...
853,2023-12-26,japones 2,Alimentação especial,-174.00
854,2023-12-28,benefício salário bhub,Salário,1000.00
855,2023-12-28,supermercado,Alimentação cotidiana,-734.48
856,2023-12-29,pix transf carolin29/12,Reembolsos,53.50


## apply vectorization to description column

In [ ]:
import time
import os
from google import genai
from google.genai import types
from tqdm.auto import tqdm
from machine_learning.transactions_classification.lib.google_embedding import normalize_embeddings
tqdm.pandas()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
GEMINI_MODEL = os.getenv('GEMINI_MODEL', 'gemini-embedding-001')


In [ ]:
from machine_learning.transactions_classification.lib.google_embedding import create_embeddings_batch, embed_and_normalize_embeddings

embeddings = create_embeddings_batch(complete_dataset['Descrição'].to_list())

  0%|          | 0/35 [00:00<?, ?it/s]

100%|██████████| 35/35 [00:26<00:00,  1.33it/s]


In [ ]:
embedded_dataset = embed_and_normalize_embeddings(complete_dataset, embeddings)

In [ ]:
embedded_dataset.to_parquet(dataset_path / 'embedded_dataset.parquet')

In [ ]:
from security.dataset import encrypt_dataset
import pickle

encrypted_dataset, dtypes = encrypt_dataset(embedded_dataset, key)
encrypted_dataset.to_parquet(dataset_path / 'encrypted_embedded_dataset.parquet')
dtypes_path = dataset_path / 'dtypes_embedded_dataset.pkl'
with open(dtypes_path, 'wb') as f:
    pickle.dump(dtypes, f)
